In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from divergen import CodebaseAssistant

# CodebaseAssistant

In [ ]:
codebase_assitant = CodebaseAssistant(
    source_dir="../src/",
    prompt_manager={"prompt_library":"../assets/prompt-library"}
)

### Generating docstings

In [ ]:
codebase_assitant.generate_docstrings(entity_name="PromptManager")

In [ ]:
codebase_assitant.apply_changes()

In [ ]:
codebase_assitant.revert_changes()

In [ ]:
codebase_assitant.reject_changes()

### Explaining codebase

In [ ]:
codebase_assitant.explain_codebase(
    entity_name="CodebaseAssistant", copy_to_clipboard=True
)

# CodebaseManager

The codebase assistant inherits from this class, however, we instantiate it on its own here for demo purpose

In [ ]:
from divergen import CodebaseManager

In [ ]:
codebase_manager = CodebaseManager(source_dir="../src")

In [ ]:
codebase_manager.parse_modules()

In [ ]:
module_content = codebase_manager._modules[
    "../src/divergen/codebase_manager/codebase_manager.py"
]

In [ ]:
module_content.entities

In [ ]:
print(
    module_content.entities["CodebaseManager.get_module_path"].source_code
)

### Getting specific entity (class, method, function)

In [ ]:
print(
    codebase_manager.get_entity_source_code(entity_name="CodebaseAssistant")
)

In [ ]:
print(
    codebase_manager.get_entity_source_code(entity_name="CodebaseAssistant.explain_codebase")
)

### Getting the source code of a module

In [ ]:
print(
    codebase_manager.get_module_source_code(module_name="codebase_manager.py")
)

In [ ]:
print(
    codebase_manager.get_codebase_source_code()
)

In [ ]:
import pyperclip
pyperclip.copy(codebase_manager.get_codebase_source_code())

# Prompt manager

In [ ]:
from divergen import PromptManager

In [ ]:
prompt_manager = PromptManager(prompt_library="../assets/prompt-library")

In [ ]:
print(
    prompt_manager.build_template(
        template_name="explain-codebase.yaml", 
        template_inputs={"code":"def function: ..."}
    )
)

In [ ]:
print(
    prompt_manager.execute_template(
        template_name="explain-codebase.yaml", 
        template_inputs={"code":"def function: ..."},
        model_name="fake"
    )
)